In [1]:
val plainText = sc.textFile("/home/user/Documents/PROJETS/DATA/AirBnB/comment_fr.csv")
plainText.take(5).foreach(println)

comments
"Nous n'avons pas croisé Aurélie qui était hors de la capital aux dates où nous avons loué son appartement. Toutefois, la communication avec elle a été très simple ! 
Son appartement est idéal pour deux personnes.
Il est également très bien situé, à deux pas de l'arrêt de métro Couronnes sur la ligne 2. 
Son chat Punky nous a tenu compagnie durant notre séjour et il est très facile à vivre ! "


plainText = /home/user/Documents/PROJETS/DATA/AirBnB/comment_fr.csv MapPartitionsRDD[1] at textFile at <console>:27


/home/user/Documents/PROJETS/DATA/AirBnB/comment_fr.csv MapPartitionsRDD[1] at textFile at <console>:27

In [2]:
// Convert RDD to DF with ID for every document
val comment = plainText.zipWithIndex.toDF("comment", "id")


comment = [comment: string, id: bigint]


[comment: string, id: bigint]

In [3]:
import org.apache.spark.ml.feature.RegexTokenizer

// Set params for RegexTokenizer
val tokenizer = new RegexTokenizer()
  .setPattern("[\\W_]+")
  .setMinTokenLength(4) // Filter away tokens with length < 4
  .setInputCol("comment")
  .setOutputCol("tokens")

// Tokenize document
val tokenized_comment = tokenizer.transform(comment)

tokenizer = regexTok_bd83c66c3669
tokenized_comment = [comment: string, id: bigint ... 1 more field]


[comment: string, id: bigint ... 1 more field]

In [4]:
tokenized_comment.select("tokens").take(10)

[comments]
"[nous, avons, crois, tait, hors, capital, dates, nous, avons, appartement, toutefois, communication, avec, elle, simple]"
"[appartement, pour, deux, personnes]"
"[galement, bien, situ, deux, couronnes, ligne]"
"[chat, punky, nous, tenu, compagnie, durant, notre, jour, facile, vivre]"
"[accueil, tres, attentionn]"
"[propri, taire, disponible, able]"
"[appartement, alement, situ, entoure, commerces, tres, bien, desservi, metro]"
"[surtout, appartement, correspond, descriptif, propre]"
[recommande]


In [5]:
// List of stopwords
val stopwords = sc.textFile("/home/user/Documents/RCP-216/Projet/stopwords-fr.txt").collect()


stopwords = Array(a, abord, absolument, afin, ah, ai, aie, aient, aies, ailleurs, ainsi, ait, allaient, allo, allons, allô, alors, anterieur, anterieure, anterieures, apres, après, as, assez, attendu, au, aucun, aucune, aucuns, aujourd, aujourd'hui, aupres, auquel, aura, aurai, auraient, aurais, aurait, auras, aurez, auriez, aurions, aurons, auront, aussi, autre, autrefois, autrement, autres, autrui, aux, auxquelles, auxquels, avaient, avais, avait, avant, avec, avez, aviez, avions, avoir, avons, ayant, ayez, ayons, b, bah, bas, basee, bat, beau, beaucoup, bien, bigre, bon, boum, bravo, brrr, c, car, ce, ceci, cela, celle, celle-ci, celle-là, celles, celles-ci, celles-là, celui, celui-ci, celui-là, celà, cent, cependant, certain, certaine, certaines, certains, certes, ces...


[a, abord, absolument, afin, ah, ai, aie, aient, aies, ailleurs, ainsi, ait, allaient, allo, allons, allô, alors, anterieur, anterieure, anterieures, apres, après, as, assez, attendu, au, aucun, aucune, aucuns, aujourd, aujourd'hui, aupres, auquel, aura, aurai, auraient, aurais, aurait, auras, aurez, auriez, aurions, aurons, auront, aussi, autre, autrefois, autrement, autres, autrui, aux, auxquelles, auxquels, avaient, avais, avait, avant, avec, avez, aviez, avions, avoir, avons, ayant, ayez, ayons, b, bah, bas, basee, bat, beau, beaucoup, bien, bigre, bon, boum, bravo, brrr, c, car, ce, ceci, cela, celle, celle-ci, celle-là, celles, celles-ci, celles-là, celui, celui-ci, celui-là, celà, cent, cependant, certain, certaine, certaines, certains, certes, ces, cet, cette, ceux, ceux-ci, ceux-là, chacun, chacune, chaque, cher, chers, chez, chiche, chut, chère, chères, ci, cinq, cinquantaine, cinquante, cinquantième, cinquième, clac, clic, combien, comme, comment, comparable, comparables, co

In [6]:
import org.apache.spark.ml.feature.StopWordsRemover

// Set params for StopWordsRemover
val remover = new StopWordsRemover()
  .setStopWords(stopwords) // This parameter is optional
  .setInputCol("tokens")
  .setOutputCol("filtered")

// Create new DF with Stopwords removed
val filtered_comment = remover.transform(tokenized_comment)

remover = stopWords_f7f0d8f2d06a
filtered_comment = [comment: string, id: bigint ... 2 more fields]


[comment: string, id: bigint ... 2 more fields]

In [7]:
import org.apache.spark.ml.feature.CountVectorizer

// Set params for CountVectorizer
val vectorizer = new CountVectorizer()
  .setInputCol("filtered")
  .setOutputCol("features")
  .setVocabSize(10000)
  .setMinDF(5)
  .fit(filtered_comment)


[Stage 5:>                                                          (0 + 2) / 2]

vectorizer = cntVec_10cc9d93be3f


cntVec_10cc9d93be3f

In [8]:
// Create vector of token counts
val countVectors = vectorizer.transform(filtered_comment).select("id", "features")

countVectors = [id: bigint, features: vector]


[id: bigint, features: vector]

In [9]:
// Convert DF to RDD
import org.apache.spark.mllib.linalg.Vector
import org.apache.spark.sql.Row
val lda_countVector = countVectors.map { case Row(id: Long, countVector: Vector) => (id, countVector) }

lda_countVector = [_1: bigint, _2: vector]


[_1: bigint, _2: vector]

In [ ]:
val numTopics = 20
numTopics: Int = 20
import org.apache.spark.mllib.clustering.{LDA, OnlineLDAOptimizer}

// Set LDA params
val lda = new LDA()
  .setOptimizer(new OnlineLDAOptimizer().setMiniBatchFraction(0.8))
  .setK(numTopics)
  .setMaxIterations(3)
  .setDocConcentration(-1) // use default values
  .setTopicConcentration(-1) // use default values
val ldaModel = lda.run(lda_countVector)

In [ ]:
// Review Results of LDA model with Online Variational Bayes
val topicIndices = ldaModel.describeTopics(maxTermsPerTopic = 5)
val vocabList = vectorizer.vocabulary
val topics = topicIndices.map { case (terms, termWeights) =>
  terms.map(vocabList(_)).zip(termWeights)
}
println(s"$numTopics topics:")
topics.zipWithIndex.foreach { case (topic, i) =>
  println(s"TOPIC $i")
  topic.foreach { case (term, weight) => println(s"$term\t$weight") }
  println(s"==========")
}